In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math
import os

In [5]:
def getURL(page):
    """
    :param page: html of web page (here: Python home page) 
    :return: urls in that page 
    """
    start_link = page.find("a href")
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

def PageScraper(url):
    l=[]
    response = requests.get(url)
    page = str(BeautifulSoup(response.content))
    while url!= '#comment_form':
        url, n = getURL(page)
        page = page[n:]
        if url and url[:5]=='/book':
            l.append(url)
    return l

#Scraps the first 100 pages
outfile = open("Pages101-200.txt", "a")
for k in range(101,201):
    url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever?page='+str(k)
    L = PageScraper(url)
    for k in range(len(L)):
        outfile.write('https://www.goodreads.com/'+L[k])
        outfile.write("\n")
outfile.close()

In [10]:
#Creating folders fo each page
path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
os.chdir(path)
for i in range(101):
    if 0<i<10:
        os.makedirs("page_10"+str(i))
    if 9<i<100:
        os.makedirs("page_1"+str(i))
    if i==100:
        os.makedirs("page_200")

In [30]:
# First, I make sure i am in the directory that has the text file with the URLS
path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
os.chdir(path)

i = 9901
page = 100

with open("Pages101-200.txt", "r") as urls:
    # I previously created the 100 directories, from page 101 to page 200
    # I change the working directory for which i want the HTMLs 
    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_200"
    os.chdir(path)

    for url in urls.readlines():

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logic here is not the best but it works- 
        # Basically, once every 100 urls treated I change the directory
        
        if i <= page*100:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        if i == (page*100)+1 and 0<page<9:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_10"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            
        if i == (page*100)+1 and 8<page<99:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_1"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            
        if i == (page*100)+1 and page==99:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page*2)
            os.chdir(path)
    
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            

ValueError: unknown url type: ''

In [106]:
# SECTION 1

# All functions to retrieve the informations

def get_title(soup):
    try:
        title = soup.find(id="bookTitle").contents[0].strip()
        return title
    except AttributeError:
        return ""

def get_series(soup):
    try:
        serie = soup.find(id="bookSeries").get_text().strip().replace("(","").replace(")","")
        #regex = "#.*"
        #serie = re.sub(regex, "",serie)
        return serie
    except AttributeError:
        return ""

def get_author(soup):
    try:
        author = soup.find_all(class_="authorName__container")
        author_list = " ".join([x.get_text().strip() for x in author])
        return author_list
    except AttributeError:
        return ""

def get_pages(soup):
    try:
        pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
        return int(pages)
    except AttributeError:
        return ""

def get_ratingValue(soup):
    try:
        ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
        return float(ratingValue)
    except AttributeError:
        return ""

def get_ratingCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[0][0].replace(",",""))
    except AttributeError:
        return ""

def get_reviewCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[1][0].replace(",",""))
    except AttributeError:
        return ""

def get_date(soup):
    try:
        regex = "by.*"
        date = soup.find_all(class_ = "row")
        date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
        return date
    except AttributeError:
        return ""

def get_setting(soup):
    try:
        string = re.compile("/places/.")
        test = soup.find_all(href=string)

        places_list = " ".join([(str(a.contents[0])) for a in test])

        return places_list
    except AttributeError:
        return ""

def get_plot(soup):
    try:
        plot = soup.find(id="description")
        plot = plot.find_all(id= re.compile("freeText\.*"))

        final_plot = [] # It seems i need a list because joining on an empty string gives me a !"£$%&"@ blank

    # now there should not be a case where there are 3 span tags ...hopefully
        if len(plot) > 1:
            final_plot.append(plot[1].get_text())
        else:
            final_plot.append(plot[0].get_text())

        return final_plot[0]
    except AttributeError:
        return ""


def get_char(soup):
    try:
        string = re.compile("/characters/.")
        test = soup.find_all(href=string)

        char_list= " ".join([(str(a.contents[0])) for a in test])

        return char_list
    except AttributeError:
        return ""

def get_url(soup):
    try:
        regex = "https://www.goodreads.com/book/show/."
        ratingCount = soup.find(href =re.compile(regex), rel="canonical")
        return ratingCount.get('href')
    except AttributeError:
        return ""


In [107]:
# SECTION 2


eng_count = 1
nb_html_articles = 100
page = 101


for k in range(3):

    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page)
    os.chdir(path)
    
    
    for i in range(1,nb_html_articles+1):

        path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page)
        os.chdir(path)
    
        html = open("article_"+str(k*100+i)+".html", "r", encoding="utf-8")
        soup = BeautifulSoup(html.read(), 'html.parser')
        try:
            if detect(get_plot(soup)) == "en":
                eng_count += 1
                
                path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\Test"
                os.chdir(path)
                with open("article_"+str(eng_count)+".tsv", 'w', encoding="utf-8") as f_output:
                    test_list_tsv = [get_title(soup), 
                    get_series(soup), 
                    get_author(soup),
                    get_ratingValue(soup),
                    get_ratingCount(soup), 
                    get_reviewCount(soup),
                    get_plot(soup),
                    get_pages(soup),
                    get_date(soup),
                    get_setting(soup),
                    get_char(soup),
                    get_url(soup)]
            
                    header_list = ["bookTitle","bookSeries","bookAuthors","ratingValue","ratingCount","reviewCount","Plot","NumberofPages","Date","Characters","Setting","URL"]

                    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
                    os.chdir(path)

                    tsv_output = csv.writer(f_output, delimiter='\t')
                    tsv_output.writerow(header_list)
                    tsv_output.writerow(test_list_tsv)
                    
        except:
            language ="error"
        #ErrorCode.CantDetectError
    page = page + 1

KeyboardInterrupt: 

In [75]:
path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\Data(no 0)"
os.chdir(path)
t = []
s = []
p = []
y = []
for x in range(1,26886):
    try:
        data = pd.read_csv("article_"+str(x)+".tsv", sep = "\t", encoding = "utf-8")
        title = data['bookTitle'].item()
        series = data['bookSeries'].item()
        pages = data['NumberofPages'].item()
        date = data['Date'].item()
        year = str(date)[-4:]
        t.append(title)
        s.append(series)
        p.append(pages)
        y.append(int(year))
    except FileNotFoundError :
        pass
    

C:\Users\Eloisa\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Eloisa\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Eloisa\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
C:\Users\Eloisa\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]


In [88]:
yl = []

for i in range(len(y)):
    try:
        yl.append(int(y[i]))
    except:
        print('wrong value', y[i])
        
yl

wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong value nan
wrong va

[2008,
 2004,
 2006,
 2000,
 2006,
 2006,
 1996,
 2002,
 2012,
 2012,
 1999,
 2007,
 1964,
 2002,
 2006,
 2004,
 2005,
 2000,
 2003,
 1987,
 2011,
 2012,
 1999,
 2004,
 2014,
 1999,
 2002,
 2004,
 2007,
 2004,
 2009,
 2003,
 2003,
 2000,
 2001,
 2002,
 2013,
 1986,
 1998,
 2003,
 2001,
 2003,
 2006,
 1998,
 2007,
 2005,
 2017,
 2002,
 1997,
 2006,
 2000,
 1988,
 2006,
 2006,
 2003,
 2007,
 1995,
 1999,
 2004,
 2018,
 1998,
 2006,
 2004,
 2019,
 2002,
 1990,
 1998,
 1975,
 1998,
 2004,
 2007,
 2006,
 1963,
 2012,
 1998,
 2005,
 2005,
 2008,
 2013,
 2013,
 2019,
 2006,
 2019,
 2006,
 2002,
 2005,
 2007,
 1981,
 2005,
 1996,
 2003,
 2004,
 1973,
 1999,
 1996,
 2000,
 2001,
 1984,
 2004,
 1990,
 2003,
 1997,
 2002,
 2003,
 2010,
 2006,
 2004,
 2003,
 1989,
 1996,
 2007,
 1980,
 2006,
 2005,
 2005,
 1976,
 1972,
 2006,
 2005,
 2004,
 1994,
 2000,
 2009,
 1994,
 1999,
 2000,
 2002,
 2003,
 1994,
 2003,
 2018,
 2009,
 2003,
 1985,
 2011,
 1987,
 1999,
 2000,
 2005,
 2009,
 2005,
 1987,
 2003,

In [46]:
cl = [x for x in s if str(x) != 'nan']

In [66]:
sl = cl[:10]

In [98]:
sl

['The Hunger Games ',
 'Harry Potter ',
 'To Kill a Mockingbird',
 'The Twilight Saga ',
 'The Chronicles of Narnia Publication Order ',
 'The Lord of the Rings ',
 "The Hitchhiker's Guide to the Galaxy ",
 'Robert Langdon ',
 "Alice's Adventures in Wonderland ",
 'Divergent ']

In [99]:
dictionary_pages = {}
dictionary_years = {}
for j in range(len(sl)):
    index = []
    pages = []
    years = []
    for i in range(len(s)):
        if sl[j] == s[i]:
            index.append(i)
    for elem in index:
        pages.append(p[elem])
        years.append(y[elem])
    dictionary_pages[sl[j]] = pages
    dictionary_years[sl[j]] = years
    
            
        

In [100]:
dictionary_pages

{'The Hunger Games ': [374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  374,
  374,
  1155,
  391,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390],
 'Harry Potter ': [870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  870,
  309,
  759,
  435,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  343,
  1500,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  343,
  1500,
  3342,
  4],
 'To Kill a Mockingbird': [324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  309],
 'The Twilight Saga ': [501,
  1690,
  501,
  1690,
  178,
  629,
  756,
  563,
  501,
  169

In [101]:
dictionary_years

{'The Hunger Games ': [2008,
  '2010',
  '2009',
  '2010',
  '2008',
  '2010',
  '2009',
  '2010',
  '2008',
  '2010',
  '2009',
  '2010',
  '2008',
  '2008',
  '2008',
  '2010',
  '2009',
  '2008',
  '2010',
  '2009',
  '2010',
  '2008',
  '2010',
  '2009',
  '2010',
  '2008',
  '2010',
  '2009',
  '2010',
  '2008',
  '2010',
  '2009',
  '2010'],
 'Harry Potter ': [2004,
  2003,
  2007,
  2004,
  2002,
  2006,
  '1999',
  '2007',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2007',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2007',
  '2004',
  '2004',
  '2003',
  '2007',
  '2004',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2007',
  '2016',
  '1999',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2007',
  '2004',
  '2003',
  '2007',
  '2004',
  '2002',
  '2006',
  '1999',
  '2007',
  '2004',
  '2003',
  '20

In [70]:
Y = [] #Stores the nb_pages for all 10 bookseries

for k in range(len(sl)):
    
    yaxis = [dictionary_pages[sl[k]][0]] #initialize y with the nb_pages of the first book
    
    for i in range(1,len(dictionary_pages[sl[k]])):
        
        yaxis.append(dictionary_pages[sl[k]][i] + yaxis[i-1]) #Add the cumulative nb_pages to y
        
    Y.append(yaxis) 

In [71]:
Y

[[374,
  1529,
  1920,
  2310,
  2684,
  3839,
  4230,
  4620,
  4994,
  6149,
  6540,
  6930,
  7304,
  7678,
  8052,
  9207,
  9598,
  9972,
  11127,
  11518,
  11908,
  12282,
  13437,
  13828,
  14218,
  14592,
  15747,
  16138,
  16528,
  16902,
  18057,
  18448,
  18838],
 [870,
  1179,
  1938,
  2373,
  3107,
  3759,
  4100,
  8200,
  9070,
  9379,
  10138,
  10573,
  11307,
  11959,
  12300,
  16400,
  17270,
  17579,
  18338,
  18773,
  19507,
  20159,
  20500,
  24600,
  25470,
  26340,
  26649,
  27408,
  27843,
  28713,
  29022,
  29781,
  30216,
  30950,
  31602,
  31943,
  32813,
  33122,
  33881,
  34316,
  35050,
  35702,
  36043,
  40143,
  40486,
  41986,
  42856,
  43165,
  43924,
  44359,
  45093,
  45745,
  46086,
  50186,
  51056,
  51365,
  52124,
  52559,
  53293,
  53945,
  54286,
  58386,
  59256,
  59565,
  60324,
  60759,
  61493,
  62145,
  62486,
  66586,
  66929,
  68429,
  71771,
  71775],
 [324, 648, 972, 1296, 1620, 1944, 2268, 2592, 2916, 3240, 3549],

In [49]:
dictionary_pages

{'The Hunger Games ': [374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  374,
  374,
  1155,
  391,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390,
  374,
  1155,
  391,
  390],
 'Harry Potter ': [870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  870,
  309,
  759,
  435,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  343,
  1500,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  870,
  309,
  759,
  435,
  734,
  652,
  341,
  4100,
  343,
  1500,
  3342,
  4],
 'To Kill a Mockingbird': [324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  324,
  309],
 'The Twilight Saga ': [501,
  1690,
  501,
  1690,
  178,
  629,
  756,
  563,
  501,
  169

In [23]:
dictionary

{'The Hunger Games ': [0,
  185,
  222,
  321,
  742,
  927,
  964,
  1063,
  1996,
  2181,
  2218,
  2317,
  2906,
  2914,
  3014,
  3199,
  3236,
  3291,
  3476,
  3513,
  3613,
  5179,
  5364,
  5401,
  5501,
  6119,
  6304,
  6341,
  6441,
  7166,
  7351,
  7388,
  7488],
 'Harry Potter ': [1,
  31,
  70,
  91,
  102,
  105,
  125,
  406,
  743,
  773,
  812,
  833,
  844,
  847,
  867,
  1148,
  1997,
  2027,
  2066,
  2087,
  2098,
  2101,
  2121,
  2402,
  2907,
  2915,
  2945,
  2984,
  3005,
  3015,
  3045,
  3084,
  3105,
  3116,
  3119,
  3139,
  3292,
  3322,
  3361,
  3382,
  3393,
  3396,
  3416,
  3698,
  4601,
  4867,
  5180,
  5210,
  5249,
  5270,
  5281,
  5284,
  5304,
  5586,
  6120,
  6150,
  6189,
  6210,
  6221,
  6224,
  6244,
  6526,
  7167,
  7197,
  7236,
  7257,
  7268,
  7271,
  7291,
  7573,
  8476,
  8743,
  9408,
  16904],
 'To Kill a Mockingbird': [2,
  744,
  1998,
  2908,
  2916,
  3016,
  3293,
  5181,
  6121,
  7168,
  24998],
 'The Twilight Saga '

In [25]:
for i in range(len(dictionary)):
    dictionary.get(i)